Zieht sich die Parameterwerte aller Durchläufe in einen pandas dataframe

In [1]:
import pandas as pd
import glob
import numpy as np

#Dataset with or without manipulation?
manip = 1

path =r'./data/newdata' # use your path
allFiles = glob.glob(path + "/*.txt")
frame = pd.DataFrame()

#will be converted to dataframe
list_ = []

#dict of style index in frame : (filename, index in file)
findfile = {}

#index for rows in frame
curr_row = 0
for file_ in allFiles:
    orig_ind = 0
    n = 3 + manip  #Add number of lines that will appear if manipulation is on!
    num_lines = sum(1 for l in open(file_))
    skip_idx = [x for x in range(1, num_lines) if x % n != 0]
    df = pd.read_csv(file_,index_col=None, skiprows=skip_idx, header = None)
    list_.append(df)
    for i in range(curr_row, curr_row+len(df.index)):
        findfile[i] = (file_, orig_ind)
        orig_ind += 1
    curr_row +=len(df.index)
frame = pd.concat(list_, ignore_index=True)
#with pd.option_context('display.max_rows', 100, 'display.max_columns', 20):
#    display(frame)

Funktion die für gegebenes NNodes, NEdges, phi den Index im Datenframe (der Parameterwerte) ausgibt

In [2]:
import re

def getIndex(NNodes, NEdges, phi, kmanip):
    lines = []
    for j in range(frame.shape[0]): #all rows
        getstr = frame[0][j]
        if ([int(s) for s in getstr.split() if s.isdigit()][0] == NNodes):
            getstr = frame[1][j]
            if ([int(s) for s in getstr.split() if s.isdigit()][0] == NEdges):
                getstr = frame[2][j]
                if (float(re.findall("\d+\.\d+", getstr)[0]) == phi):
                    if manip != 1:
                        lines.append(j)
                    else:
                        getstr = frame[4][j]
                        if ([int(s) for s in getstr.split() if s.isdigit()][0] == kmanip):
                            lines.append(j)
    return lines

Funktion die für gegebenen Index des Dataframes (der Parameterwerte) den Meinungsvektor (OporComp = 0) oder den Komponentenvektor (OporComp = 1) oder den Influencervektor (OporComp = 2) aus der jeweiligen Datei liest

In [3]:
#OporComp = 0 for OpinionVector, 1 for ConnectedComponents
def getVector(frameIndex, OporComp):
    filename = (findfile[frameIndex])[0]
    fileindex = (findfile[frameIndex])[1]
    fileline = fileindex*(3+manip)+1
    OpinionVector = pd.read_csv(filename, skiprows = fileline+OporComp, nrows = 1, header=None, delim_whitespace = True)
    return OpinionVector

Funktion die für gegebenes NNodes, NEdges, phi einen Matplotlib subplot eines Plot der Größe x * y an index z erstellt

In [4]:
from collections import Counter
import matplotlib.pyplot as plt


#ComponentSizes dependent on Phi
def histogram( NNodes, phi, kinf):
    hist = [0] * (NNodes+1)
    lines = []
    lines = getIndex(NNodes, NNodes*2, phi, kinf)
    if (len(lines) == 0):
        print("No Dataset found for NNodes = %d, phi = %.3f" %(NNodes,  phi))
        return
    for i in range(len(lines)):
        compvec=[NNodes] #empty array
        compvec = getVector(lines[i],1) #getComponentsizevector
        compvec = compvec.values.tolist()[0] #get list from dataframe
        compdic = Counter(compvec) #get Dict of type (compent number) : size
        comps = list(compdic.values()) #get list of size values
        d = Counter(comps) #get Dict sizeofCompononent : appearance
        for siz, app in d.items():
            hist[siz] += app
        hist[:] = [float(x) / NNodes for x in hist]
    hist[:] = [float(x) / len(lines) for x in hist] #average over number of datasets
    #hist[:] = [float(x) / NNodes for x in hist] 
    #axs = axs.flatten()
    #axs[ithplot].loglog(hist, 'o')
    plt.loglog(hist, 'o')
    title = "Phi: " +str(float(phi)) + " averaged over " + str(len(lines))
    #axs[ithplot].set_title(title)
    #axs[ithplot].set_xlim(xmin=0, xmax=NNodes)
    #axs[ithplot].set_ylim(ymin = 0.00001, ymax=1.1)
    plt.title(title)
    plt.xlim(xmin=0, xmax=NNodes)
    plt(ylimymin = 0.00001, ymax = 1.1)
    plt.show()
    picfilename = input("Filename? 0 to cancel")
    if picfilename != '0':
        picfilename = "./hist/" +str(picfilename)
        fig.savefig(picfilename) 

In [5]:
def WhichNNodes():
    Nodelist = []
    for j in range(frame.shape[0]):
        getstr = frame[0][j]
        newNode = [int(s) for s in getstr.split() if s.isdigit()][0]
        if newNode not in Nodelist:
            Nodelist.append(newNode)
    return Nodelist

def WhichPhi():
    philist = []
    for j in range(frame.shape[0]): #all rows
        getstr = frame[2][j]
        newphi = float(re.findall("\d+\.\d+", getstr)[0])
        if newphi not in philist:
            philist.append(newphi)
    return philist

def WhichKinf():
    KinfList = []
    for j in range(frame.shape[0]): #all rows
        getstr = frame[4][j]
        newkinf = [int(s) for s in getstr.split() if s.isdigit()][0]
        if newkinf not in KinfList:
            KinfList.append(newkinf)
    return KinfList


DomOp gets Values for NNodes and phi and returns the ratio of Opinion 0 Nodes and other Opinion nodes

In [6]:
def DomOp(NNodes, phi, kInf):
    Dom = 0.
    lines = []
    lines = getIndex(NNodes, NNodes*2, phi, kInf)
    
    if (len(lines) == 0):
        print("No Dataset found for %d, %d, %.3f, %d" %(NNodes, NNodes*2, phi, kInf)) 
        return -1
    
    for i in range(len(lines)):        
        #get opinion vector
        opvec = getVector(lines[i],0)
        
        #get list from dataframe
        opvec = opvec.values.tolist()[0]
        
        #get Dict of type (opinion number) : size
        opvec = Counter(opvec) 
        #print(opvec)
        #get Number of Nodes that have Opinion 0
        Curr_Dom = float(list(opvec.values())[0])
        #print(Curr_Dom)
        #divided by mean of all OTHER opinions
        Curr_Dom = Curr_Dom/(NNodes - Curr_Dom) * (NNodes/10 -1) 
        #print(Curr_Dom)
        #print(CurrInfOpDom)
        Dom += Curr_Dom
        
    Dom = Dom/(len(lines))
    return Dom

Gets NNodes value and InPhi, InNinf list and creates plot of OpDomination / Phi with a line for each element of InNinf

In [7]:
def Plot_DomPhi(NNodes, InPhi, InNinf):
    plotdata = [[0 for x in range(len(InPhi))] for y in range(len(InNinf))] 
    for i, Phi in enumerate(InPhi):
        for j, Inf in enumerate(InNinf):
            temp = DomOp(NNodes, Phi, Inf)
            if temp != -1:
                plotdata[i][j] = DomOp(NNodes, Phi, Inf)
        plt.scatter(InPhi, plotdata[i])
    plt.show()
    picfilename = input("Filename? 0 to cancel")
    if picfilename != '0':
        picfilename = "./DomPhi/" +str(picfilename)
        fig.savefig(picfilename) 
    return


def Plot_DomKinf(NNodes, InPhi, InNinf):
    plotdata = [[0 for x in range(len(InNinf))] for y in range(len(InPhi))] # plotdata[i][j] limited by i<len(InPhi) and j<len(InNinf)!!!!
    for i,Inf in enumerate(InNinf):
        for j,Phi in enumerate(InPhi):
            temp = DomOp(NNodes, Phi, Inf)
            if temp != -1:
                plotdata[j][i] = temp
            else:
                continue
    return plotdata
    #    plt.scatter(InNinf, plotdata[i])
    #plt.show()
    #picfilename = input("Filename? 0 to cancel")
    #if picfilename != '0':
    #    picfilename = "./DomKinf/" +str(picfilename)
    #    fig.savefig(picfilename) 

    
def Plot_DomKinf2(NNodes, InPhi, InNinf):
    
    # plotdata[i][j] limited by i<len(InPhi) and j<len(InNinf)!!!!
    for i,Inf in enumerate(InNinf):
        for j,Phi in enumerate(InPhi):
            temp = DomOp(NNodes, Phi, Inf)
            if temp != -1:
                plotdata[j][i] = temp
            else:
                continue
    return plotdata

In [ ]:
getdata = Plot_DomKinf2(WhichNNodes()[0],WhichPhi(), WhichKinf() ) #getdata ist 2D Array InPhi x InNinf mit DomOp Werten

In [ ]:
from tempfile import TemporaryFile
outfile = TemporaryFile()
np.save(outfile, getdata)

In [ ]:
print(getdata)

In [ ]:
x = WhichPhi()
y = WhichKinf()

xs, ys = np.meshgrid(x, y)
fig = plt.figure()
ax = Axes3D(fig)
ax.plot_surface(getdata, rstride=1, cstride=1, cmap='hot')
plt.show(